In [1]:
import torch
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    ShortSideScale,
    UniformTemporalSubsample,
    Div255,
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    Resize,
    RandomHorizontalFlip,
)
from torch.utils.data import DataLoader
import torch.nn as nn
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"


%cd / workspace / Walk_Video_PyTorch / project


[Errno 2] No such file or directory: '/ workspace / Walk_Video_PyTorch / project'
/workspace/Walk_Video_PyTorch/project/misc


In [2]:
from pytorch_lightning import seed_everything
from torchmetrics.classification import BinaryAccuracy
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from dataloader.data_loader import WalkDataModule

from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()


seed_everything(42, workers=True)


ModuleNotFoundError: No module named 'models'

In [ ]:
from parameters import get_parameters

VIDEO_LENGTH = ['1', '2', '3']
VIDEO_FRAME = ['8', '16', '32']

opt, _ = get_parameters()
opt.num_workers = 8
opt.batch_size = 8
opt.gpu_num = 1

opt.version = '1201_1_16'
opt.model = "resnet"
opt.model_depth = 50
opt.model_class_num = 1

# opt.clip_duration = 2
# opt.uniform_temporal_subsample_num = 30
opt.version = opt.version + '_' + opt.model + '_depth' + str(opt.model_depth)

opt.fusion_method = 'slow_fusion'
opt.fix_layer = 'all'
# opt.train_path = '/workspace/data/split_pad_dataset_512/fold3/'

opt.transfor_learning = True
opt.pre_process_flag = True

DATA_PATH = opt.split_pad_data_path


In [ ]:
opt.train_path = os.path.join(DATA_PATH, 'flod2')

opt.clip_duration = int(1)
opt.uniform_temporal_subsample_num = int(8)

# ckpt_path = '/workspace/Walk_Video_PyTorch/logs/resnet/1119_1_8_resnet_depth50/flod2/checkpoints/epoch=11-val_loss=0.63-val_acc=0.8926.ckpt'
ckpt_path = '/workspace/Walk_Video_PyTorch/logs/resnet/1119_1_8_resnet_depth50/flod3/checkpoints/epoch=23-val_loss=0.47-val_acc=0.8568.ckpt'

print('#' * 50)
print('the data path: %s' % opt.train_path)
print('ckpt: %s' % ckpt_path)
model = WalkVideoClassificationLightningModule(opt).load_from_checkpoint(ckpt_path)

data_module = WalkDataModule(opt)
data_module.setup()
test_data = data_module.val_dataloader()


In [ ]:
transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    # uniform clip T frames from the given n sec video.
                    UniformTemporalSubsample(8),

                    # dived the pixel from [0, 255] tp [0, 1], to save computing resources.
                    Div255(),
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),

                    Resize(size=[224, 224]),
                    RandomHorizontalFlip(p=0.5),
                ]
            ),
        ),
    ]
)


In [ ]:
from data_loader import WalkDataset
from pytorchvideo.data import make_clip_sampler

dset = WalkDataset(
    data_path='/workspace/Walk_Video_PyTorch/project/misc/test_video/',
    clip_sampler=make_clip_sampler('uniform', 1),
    video_sampler=torch.utils.data.SequentialSampler,
    transform=transform,
)

test_data = DataLoader(
    dset,
    batch_size=300
)


In [ ]:
batch = next(iter(test_data))

video = batch['video'].detach()  # b, c, t, h, w
label = batch['label'].detach()  # b, class num
label = [1] * len(label)
name = batch['video_name']

# video.shape, label, name, len(name)


In [ ]:
# pred the result
model = model.cuda()
video = video.cuda()

with torch.no_grad():
    pred = model(video)

pred = pred.cpu()


In [ ]:
pred_list = []

for i in torch.sigmoid(pred):
    if i > 0.5:
        pred_list.append(1)
    else:
        pred_list.append(0)


In [ ]:
# pred_list, label,
len(label)


In [ ]:
# calc acc
acc = BinaryAccuracy(threshold=0.5)

acc(torch.sigmoid(pred).squeeze(), torch.tensor(label))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

number_list = []

for i, _ in enumerate(name):
    number = _.split('-')[1].split('.')[0]
    number_list.append(number)

category_names = list(set(number_list))
category_names.sort()

results = {
    '': list(Counter(number_list).values())
}


def survey(results, category_names, pred: list, label: list, video_num: int):
    """
    Parameters
    ----------
    results : dict
        A mapping from question labels to a list of answers per category.
        It is assumed all lists contain the same number of entries and that
        it matches the length of *category_names*.
    category_names : list of str
        The category labels.
    """
    labels = list(results.keys())
    data = np.array(list(results.values()))
    data_cum = data.cumsum(axis=1)
    category_colors = plt.colormaps['RdYlGn'](
        np.linspace(0.15, 0.85, data.shape[1]))

    fig, ax = plt.subplots(nrows=2,
                           height_ratios=[1.5, 0.5],
                           gridspec_kw={'height_ratios': [1.5, 0.5]})
    ax[1].invert_yaxis()
    ax[1].set_xlim(0, np.sum(data, axis=1).max())

    for i, (colname, color) in enumerate(zip(category_names, category_colors)):
        print((colname, color))
        widths = data[:, i]
        starts = data_cum[:, i] - widths

        rects = ax[1].barh(labels, widths, left=starts, height=0.1,
                           label=colname, color=color)

        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.5 else 'darkgrey'

        ax[1].bar_label(rects, label_type='center', color=text_color)

    ax[1].set_title('video number')

    # 生成图形
    ax[0].plot(video_number, pred, 'bo:', label='predict label', linewidth=2)  # 颜色绿色，点形圆形，线性虚线，设置图例显示内容，线条宽度为2
    ax[0].plot(video_number, label, 'r.', label='true label', linewidth=1)

    # ax[0].ylabel('label') # 横坐标轴的标题
    # ax[0].xlabel('video number') # 纵坐标轴的标题
    # ax[0].xticks(range(0, len(label))) # 设置横坐标轴的刻度为 0 到 10 的数组
    # ax[0].ylim([-0.25, 1.25]) # 设置纵坐标轴范围为 -2 到 2
    ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)  # 显示图例, 图例中内容由 label 定义
    ax[0].set_title('%s positive rate' % name[0].split('-')[0])  # 图形的标题
    ax[0].set_ylabel('label')
    ax[0].autoscale(enable=True, axis="x", tight=True)
    fig.subplots_adjust(hspace=0.5)

    return fig, ax


# 生成数据
video_number = range(0, len(label))
pred = pred_list
label = label

fig, ax = survey(results, category_names)

# 生成图形
ax[0].plot(video_number, pred, 'bo:', label='predict label', linewidth=2)  # 颜色绿色，点形圆形，线性虚线，设置图例显示内容，线条宽度为2
ax[0].plot(video_number, label, 'r.', label='true label', linewidth=1)

# ax[0].ylabel('label') # 横坐标轴的标题
# ax[0].xlabel('video number') # 纵坐标轴的标题
# ax[0].xticks(range(0, len(label))) # 设置横坐标轴的刻度为 0 到 10 的数组
# ax[0].ylim([-0.25, 1.25]) # 设置纵坐标轴范围为 -2 到 2
ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)  # 显示图例, 图例中内容由 label 定义
ax[0].set_title('%s positive rate' % name[0].split('-')[0])  # 图形的标题
ax[0].set_ylabel('label')
ax[0].autoscale(enable=True, axis="x", tight=True)
fig.subplots_adjust(hspace=0.5)

plt.show()
